In [ ]:
from transformers import pipeline

/Users/henryzelenak/Desktop/Polytechnic Classes/Spring 2025/Capstone/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


# Zero Shot

In [2]:
# Load the fact-checking pipeline using the pretrained model https://huggingface.co/fractalego/fact-checking
fact_checker = pipeline("text2text-generation", model="fractalego/fact-checking")

# Define a claim to be fact-checked
claim = "The Earth is flat."

# Get the fact-checking result
result = fact_checker(claim)

# Print the result
print("Fact-Checking Result:", result)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/790 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

Device set to use cpu
The model 'GPT2LMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration']

Fact-Checking Result: [{'generated_text': 'The Earth is flat. It is composed of six planets and four main planets, many of which are double stars. The International'}]


# Example from HuggingFace
https://huggingface.co/fractalego/fact-checking

# fact_checking not available.

In [ ]:
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
)

from fact_checking import FactChecker

_evidence = """
Justine Tanya Bateman (born February 19, 1966) is an American writer, producer, and actress . She is best known for her regular role as Mallory Keaton on the sitcom Family Ties (1982 -- 1989). Until recently, Bateman ran a production and consulting company, SECTION 5 . In the fall of 2012, she started studying computer science at UCLA.
"""

_claim = 'Justine Bateman is a poet.'

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
fact_checking_model = GPT2LMHeadModel.from_pretrained('fractalego/fact-checking')
fact_checker = FactChecker(fact_checking_model, tokenizer)
is_claim_true = fact_checker.validate(_evidence, _claim)

print(is_claim_true)


# Alternative Model

Define Evidence and Claim:
You specify your evidence (_evidence) and your claim (_claim).

* Load the NLI Model and Tokenizer:
The code uses AutoTokenizer and AutoModelForSequenceClassification to load the "roberta-large-mnli" model. This model has been fine-tuned for natural language inference, meaning it can determine the relationship between two sentences.

* Encode the Input Pair:
The tokenizer processes the evidence and claim together as a pair. The model then receives this pair as input and produces logits for three classes: contradiction, neutral, and entailment.

* Compute Probabilities:
The raw logits are converted to probabilities using the softmax function.

* Interpretation:

The output probabilities are mapped to the labels.
If the probability for “entailment” is higher than for “contradiction” (or you could apply a more nuanced threshold), then the evidence is considered to support the claim.
Otherwise, the evidence does not support the claim.

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Define the evidence and the claim.
_evidence = "Jane writes code for Huggingface."
_claim = "Jane is an engineer."

# Load the pretrained NLI model and tokenizer.
# The "roberta-large-mnli" model has been widely used for natural language inference tasks.
tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")
model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli")

# Encode the evidence (premise) and the claim (hypothesis) as a pair.
inputs = tokenizer(_evidence, _claim, return_tensors="pt", truncation=True)

# Get model outputs (logits).
outputs = model(**inputs)
logits = outputs.logits

# The output logits correspond to the three classes:
#   Label 0: Contradiction
#   Label 1: Neutral
#   Label 2: Entailment
# Compute probabilities with softmax.
probs = torch.softmax(logits, dim=1)[0]

# Map the probabilities to the labels.
labels = ["contradiction", "neutral", "entailment"]
result = {labels[i]: float(probs[i]) for i in range(len(labels))}

print("Fact-checking results:")
print(result)

# Interpret the results:
# If the probability for entailment is highest, we say the evidence supports the claim.
if result["entailment"] > result["contradiction"]:
    print("Conclusion: The evidence supports the claim.")
else:
    print("Conclusion: The evidence does not support the claim.")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Fact-checking results:
{'contradiction': 0.3435172736644745, 'neutral': 0.6514946818351746, 'entailment': 0.0049880752339959145}
Conclusion: The evidence does not support the claim.


# Fact Checking Paper
https://www.sciencedirect.com/science/article/pii/S0950705122006323

* **Semantic Similarity:**
Compare an input claim against a small database of fact-checked claims using sentence embeddings and cosine similarity. This step retrieves the most similar (i.e., most relevant) fact-checked claim.

* **Natural Language Inference:**
Feed the retrieved fact-checked claim (as the premise) and the input claim (as the hypothesis) into an NLI model to see if the fact-checked claim entails (supports) the input claim. If the entailment probability is high, we conclude that the input claim is supported by verified information; otherwise, it is not.

## Model 1

In [6]:
# Install necessary packages (if not already installed)
# !pip install sentence-transformers transformers

from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# -------------------------------
# Semantic Similarity Module
# -------------------------------
# Load a pre-trained Sentence Transformer model for embeddings.
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Define a small database of fact-checked claims.
fact_checked_claims = [
    "The Earth is round.",
    "Vaccines do not cause autism.",
    "Climate change is primarily human-caused."
]

# Compute embeddings for the fact-checked claims.
fact_embeddings = embedder.encode(fact_checked_claims, convert_to_tensor=True)

# Input claim to be fact-checked.
input_claim = "The Earth is flat."

# Compute the embedding for the input claim.
input_embedding = embedder.encode(input_claim, convert_to_tensor=True)

# Calculate cosine similarity between the input claim and each fact-checked claim.
cosine_scores = util.cos_sim(input_embedding, fact_embeddings)

# Retrieve the most similar fact-checked claim.
top_k = 1
top_result = torch.topk(cosine_scores, k=top_k)
top_index = top_result.indices[0].item()

print("Most similar fact-checked claim:", fact_checked_claims[top_index])
print("Cosine similarity score:", cosine_scores[0][top_index].item())

# -------------------------------
# Natural Language Inference Module
# -------------------------------
# Load an NLI model and tokenizer (using roberta-large-mnli).
nli_tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")
nli_model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli")

# For NLI, treat the retrieved fact-checked claim as the premise and the input claim as the hypothesis.
premise = fact_checked_claims[top_index]
hypothesis = input_claim

# Tokenize the pair.
nli_input = nli_tokenizer.encode_plus(premise, hypothesis, return_tensors="pt", truncation=True)

# Get the model's output logits.
nli_output = nli_model(**nli_input)
logits = nli_output.logits

# Convert logits to probabilities.
probabilities = torch.softmax(logits, dim=1)[0]
nli_labels = ["contradiction", "neutral", "entailment"]
print("NLI probabilities:", {nli_labels[i]: probabilities[i].item() for i in range(3)})

# -------------------------------
# Decision and Conclusion
# -------------------------------
# We interpret the result as follows:
# If the entailment probability (index 2) is the highest, then the evidence supports the claim.
if probabilities[2] > probabilities[0] and probabilities[2] > probabilities[1]:
    print("Conclusion: The input claim is supported by the fact-checked claim (i.e., it is likely true).")
else:
    print("Conclusion: The input claim is not supported by the fact-checked claim (i.e., it is likely false).")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Most similar fact-checked claim: The Earth is round.
Cosine similarity score: 0.7745267152786255


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


NLI probabilities: {'contradiction': 0.9992362260818481, 'neutral': 0.00043171714060008526, 'entailment': 0.00033214010181836784}
Conclusion: The input claim is not supported by the fact-checked claim (i.e., it is likely false).


## Model 2

In [18]:
# from sentence_transformers import SentenceTransformer, util
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import torch

# # -------------------------------
# # Semantic Similarity Module using S-BERT
# # -------------------------------
# # Load a S-BERT model (here, "all-mpnet-base-v2") for computing embeddings.
# sbert_model = SentenceTransformer('all-mpnet-base-v2')

# # Define a small database of fact-checked claims.
# fact_checked_claims = [
#     "The Earth is round.",
#     "Vaccines do not cause autism.",
#     "Climate change is primarily human-caused."
# ]

# # Compute embeddings for the fact-checked claims.
# fact_embeddings = sbert_model.encode(fact_checked_claims, convert_to_tensor=True)

# # Input claim to be fact-checked.
# input_claim = "The Earth is flat."

# # Compute the embedding for the input claim.
# input_embedding = sbert_model.encode(input_claim, convert_to_tensor=True)

# # Calculate cosine similarity between the input claim and each fact-checked claim.
# cosine_scores = util.cos_sim(input_embedding, fact_embeddings)

# # Retrieve the most similar fact-checked claim.
# top_k = 1
# top_result = torch.topk(cosine_scores, k=top_k)
# top_index = top_result.indices[0].item()

# print("Most similar fact-checked claim:", fact_checked_claims[top_index])
# print("Cosine similarity score:", cosine_scores[0][top_index].item())

# # -------------------------------
# # Natural Language Inference (NLI) Module
# # -------------------------------
# # Load an NLI model and tokenizer (using roberta-large-mnli).
# nli_tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")
# nli_model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli")

# # Use the retrieved fact-checked claim as the premise and the input claim as the hypothesis.
# premise = fact_checked_claims[top_index]
# hypothesis = input_claim

# # Tokenize the pair for the NLI model.
# nli_inputs = nli_tokenizer.encode_plus(premise, hypothesis, return_tensors="pt", truncation=True)

# # Get the model's output logits.
# nli_outputs = nli_model(**nli_inputs)
# logits = nli_outputs.logits

# # Convert logits to probabilities.
# probs = torch.softmax(logits, dim=1)[0]
# nli_labels = ["contradiction", "neutral", "entailment"]
# print("NLI probabilities:", {nli_labels[i]: probs[i].item() for i in range(3)})

# # -------------------------------
# # Decision and Conclusion
# # -------------------------------
# # If entailment (index 2) is the most likely relation, we say the input claim is supported.
# if probs[2] > probs[0] and probs[2] > probs[1]:
#     print("Conclusion: The input claim is supported by the fact-checked claim (i.e., it is likely true).")
# else:
#     print("Conclusion: The input claim is not supported by the fact-checked claim (i.e., it is likely false).")


In [7]:
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# -------------------------------
# Semantic Similarity Module using S-BERT
# -------------------------------
# Load a S-BERT model (here, "all-mpnet-base-v2") for computing embeddings.
sbert_model = SentenceTransformer('all-mpnet-base-v2')

# Define a small database of fact-checked claims.
fact_checked_claims = [
    "The Earth is round.",
    "Vaccines do not cause autism.",
    "Climate change is primarily human-caused."
]

# Compute embeddings for the fact-checked claims.
fact_embeddings = sbert_model.encode(fact_checked_claims, convert_to_tensor=True)

# Input claim to be fact-checked.
input_claim = "The Earth is flat."

# Compute the embedding for the input claim.
input_embedding = sbert_model.encode(input_claim, convert_to_tensor=True)

# Calculate cosine similarity between the input claim and each fact-checked claim.
cosine_scores = util.cos_sim(input_embedding, fact_embeddings)

# Retrieve the top k candidates (if available, otherwise fewer).
top_k = 2
# Get the indices and scores of the top candidates.
top_results = torch.topk(cosine_scores, k=top_k)
indices = top_results.indices[0].tolist()  # Convert tensor to list

print("Top candidates based on cosine similarity:")
for i in indices:
    score = cosine_scores[0][i].item()
    print(f"Candidate Claim: '{fact_checked_claims[i]}' with Score: {score:.4f}")

# -------------------------------
# Natural Language Inference (NLI) Module
# -------------------------------
# Load an NLI model and tokenizer (using roberta-large-mnli).
nli_tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")
nli_model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli")
nli_labels = ["contradiction", "neutral", "entailment"]

print("\nNLI evaluation for each candidate:")
for i in indices:
    premise = fact_checked_claims[i]
    hypothesis = input_claim
    # Tokenize the pair for the NLI model.
    nli_inputs = nli_tokenizer.encode_plus(premise, hypothesis, return_tensors="pt", truncation=True)
    # Get the model's output logits.
    nli_outputs = nli_model(**nli_inputs)
    logits = nli_outputs.logits
    # Convert logits to probabilities.
    probs = torch.softmax(logits, dim=1)[0]
    probs_dict = {nli_labels[j]: probs[j].item() for j in range(3)}
    print(f"\nCandidate: '{premise}'")
    print("NLI probabilities:", probs_dict)
    # Simple decision based on the entailment probability.
    if probs[2] > probs[0] and probs[2] > probs[1]:
        print("Conclusion: The candidate supports the input claim.")
    else:
        print("Conclusion: The candidate does not support the input claim.")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Top candidates based on cosine similarity:
Candidate Claim: 'The Earth is round.' with Score: 0.7499
Candidate Claim: 'Climate change is primarily human-caused.' with Score: 0.2042


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



NLI evaluation for each candidate:

Candidate: 'The Earth is round.'
NLI probabilities: {'contradiction': 0.9992362260818481, 'neutral': 0.00043171714060008526, 'entailment': 0.00033214010181836784}
Conclusion: The candidate does not support the input claim.

Candidate: 'Climate change is primarily human-caused.'
NLI probabilities: {'contradiction': 0.9791197180747986, 'neutral': 0.017766743898391724, 'entailment': 0.0031135266181081533}
Conclusion: The candidate does not support the input claim.


**Principal Component Analysis (PCA)** is commonly used to reduce the dimensionality of embeddings. In the context of the referenced figure (“Diagram showing the three components of the FacTeR-Check architecture”), PCA is applied after concatenating multiple embedding models, thereby removing redundant features and compressing the combined embeddings into a lower-dimensional space. This helps:

* Speed Up Similarity Computations – Cosine similarity runs faster on smaller vectors.
* Reduce Overfitting/Noise – Sometimes certain embedding dimensions capture noise rather than meaningful semantic information.
Combine Multiple Models – If multiple embeddings (from different models) are concatenated, PCA can merge them into a single compact representation.
* This example uses scikit-learn’s PCA to transform both the fact-checked claim embeddings and the input claim embedding.

In [8]:
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.decomposition import PCA
import torch
import numpy as np

###############################################################################
# 1) Define Multiple Models
###############################################################################
model_names = [
    "all-MiniLM-L6-v2",   # Model A
    "all-mpnet-base-v2"  # Model B
]
models = [SentenceTransformer(name) for name in model_names]

def get_combined_embeddings(texts, models):
    """
    For each text in 'texts', compute embeddings from each model in 'models',
    concatenate them into one vector, and return the stacked array.
    """
    all_embs = []
    for text in texts:
        # Each model returns a 1D NumPy array (or PyTorch tensor) for the text.
        # We'll convert them to NumPy and concatenate along axis=0.
        emb_list = [m.encode(text) for m in models]  # shape e.g. [(384,), (768,)]
        combined_emb = np.concatenate(emb_list, axis=0)  # shape (384+768,) = (1152,)
        all_embs.append(combined_emb)
    return np.stack(all_embs, axis=0)  # shape (num_texts, sum_of_dims)

###############################################################################
# 2) Prepare the Database of Fact-Checked Claims
###############################################################################
fact_checked_claims = [
    "The Earth is round.",
    "Vaccines do not cause autism.",
    "Climate change is primarily human-caused."
]

# Encode & concatenate embeddings for each fact-checked claim
fact_embeddings_np = get_combined_embeddings(fact_checked_claims, models)  # shape (3, total_dim)
fact_embeddings_torch = torch.tensor(fact_embeddings_np)

###############################################################################
# 3) Fit PCA on the Database Embeddings
###############################################################################
pca_dim = 128  # You can choose another dimension based on your needs
pca_dim = min(len(fact_checked_claims), pca_dim )
pca = PCA(n_components=pca_dim)
pca.fit(fact_embeddings_np)  # learn how to reduce the combined embedding space

# Transform the existing database embeddings to PCA space
fact_embeddings_pca = torch.tensor(pca.transform(fact_embeddings_np))

###############################################################################
# 4) Encode & Transform the New Input Claim
###############################################################################
input_claim = "The Earth is flat."

# Encode the new claim with both models and concatenate
input_embedding_np = get_combined_embeddings([input_claim], models)  # shape (1, total_dim)

# Apply the same PCA transform
input_embedding_pca = torch.tensor(pca.transform(input_embedding_np))  # shape (1, pca_dim)

###############################################################################
# 5) Top-k Similarity Search in the PCA-Reduced Space
###############################################################################
cosine_scores = util.cos_sim(input_embedding_pca, fact_embeddings_pca)  # shape (1, len(fact_checked_claims))

top_k = 2
top_results = torch.topk(cosine_scores, k=top_k)
indices = top_results.indices[0].tolist()

print("Top candidates based on multi-model PCA-reduced embeddings:")
for i in indices:
    score = cosine_scores[0][i].item()
    print(f"  Candidate: '{fact_checked_claims[i]}' | Score: {score:.4f}")

###############################################################################
# 6) NLI Module to Check Each Candidate
###############################################################################
nli_tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")
nli_model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli")
nli_labels = ["contradiction", "neutral", "entailment"]

print("\nNLI evaluation for each candidate:")
for i in indices:
    premise = fact_checked_claims[i]
    hypothesis = input_claim

    # Encode premise/hypothesis pair for the NLI model
    nli_inputs = nli_tokenizer.encode_plus(premise, hypothesis, return_tensors="pt", truncation=True)
    nli_outputs = nli_model(**nli_inputs)
    logits = nli_outputs.logits

    # Convert to probabilities
    probs = torch.softmax(logits, dim=1)[0]
    probs_dict = {nli_labels[j]: probs[j].item() for j in range(3)}

    print(f"\n  Premise:   '{premise}'")
    print(f"  Hypothesis: '{hypothesis}'")
    print("  NLI probabilities:", probs_dict)

    # Simple decision rule based on highest probability for 'entailment'
    if probs[2] > probs[0] and probs[2] > probs[1]:
        print("  Conclusion: The candidate supports the input claim.")
    else:
        print("  Conclusion: The candidate does NOT support the input claim.")


Top candidates based on multi-model PCA-reduced embeddings:
  Candidate: 'The Earth is round.' | Score: 0.9980
  Candidate: 'Climate change is primarily human-caused.' | Score: -0.4946


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



NLI evaluation for each candidate:

  Premise:   'The Earth is round.'
  Hypothesis: 'The Earth is flat.'
  NLI probabilities: {'contradiction': 0.9992362260818481, 'neutral': 0.00043171714060008526, 'entailment': 0.00033214010181836784}
  Conclusion: The candidate does NOT support the input claim.

  Premise:   'Climate change is primarily human-caused.'
  Hypothesis: 'The Earth is flat.'
  NLI probabilities: {'contradiction': 0.9791197180747986, 'neutral': 0.017766743898391724, 'entailment': 0.0031135266181081533}
  Conclusion: The candidate does NOT support the input claim.


# FEVER Dataset

In [9]:
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.5 MB/s eta 0:00:00


In [10]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.decomposition import PCA
import torch
import numpy as np

###############################################################################
# 1) Load a small subset of the FEVER dataset as our fact-checked claims database.
###############################################################################
# Load the FEVER training split (using only the first 50 examples for demonstration).
fever_dataset = load_dataset("fever", "v1.0", split="train[:50]")
# Extract the "claim" field and remove duplicates.
fact_checked_claims = list(set(fever_dataset["claim"]))
# For demonstration, we'll take at most 10 claims.
fact_checked_claims = fact_checked_claims[:10]

print("Fact-checked claims from FEVER:")
for idx, claim in enumerate(fact_checked_claims):
    print(f"  {idx+1}. {claim}")

###############################################################################
# 2) Define Multiple SentenceTransformer Models for Embedding Extraction
###############################################################################
# We use two models to obtain complementary representations.
model_names = [
    "all-MiniLM-L6-v2",   # Model A (384 dimensions)
    "all-mpnet-base-v2"   # Model B (768 dimensions)
]
models = [SentenceTransformer(name) for name in model_names]

def get_combined_embeddings(texts, models):
    """
    For each text in 'texts', compute embeddings from each model in 'models',
    concatenate them into one vector, and return a NumPy array of shape (num_texts, total_dim).
    """
    all_embs = []
    for text in texts:
        # Compute embeddings from each model.
        emb_list = [m.encode(text) for m in models]  # e.g. shapes: (384,), (768,)
        combined_emb = np.concatenate(emb_list, axis=0)  # shape (1152,)
        all_embs.append(combined_emb)
    return np.stack(all_embs, axis=0)

# Compute combined embeddings for our FEVER-based fact-checked claims.
fact_embeddings_np = get_combined_embeddings(fact_checked_claims, models)  # shape: (num_claims, 1152)
print(f"\nCombined embedding shape for database: {fact_embeddings_np.shape}")

###############################################################################
# 3) Apply PCA for Dimensionality Reduction
###############################################################################
# Note: With a small number of claims, n_components must be <= number of samples.
# For demonstration, we set n_components to the minimum between 5 and the number of claims.
n_samples, total_dim = fact_embeddings_np.shape
pca_dim = min(5, n_samples)  # With very few samples, we cannot reduce to a high number.
pca = PCA(n_components=pca_dim)
pca.fit(fact_embeddings_np)  # Fit PCA on the database embeddings.

# Transform database embeddings into the PCA space.
fact_embeddings_pca = torch.tensor(pca.transform(fact_embeddings_np))

###############################################################################
# 4) Process a New Input Claim
###############################################################################
input_claim = "The Earth is round."

# Compute and concatenate the embeddings for the input claim.
input_embedding_np = get_combined_embeddings([input_claim], models)  # shape: (1, 1152)
# Transform the input claim embedding using the same PCA.
input_embedding_pca = torch.tensor(pca.transform(input_embedding_np))  # shape: (1, pca_dim)

###############################################################################
# 5) Top-k Similarity Search in the PCA-Reduced Space
###############################################################################
# Compute cosine similarity between the PCA-transformed input claim and each PCA-transformed fact-checked claim.
cosine_scores = util.cos_sim(input_embedding_pca, fact_embeddings_pca)  # shape: (1, num_claims)

# Retrieve the top k candidates (if available).
top_k = 5
top_results = torch.topk(cosine_scores, k=top_k)
indices = top_results.indices[0].tolist()  # List of candidate indices

print("\nTop candidates based on PCA-reduced embeddings:")
for i in indices:
    score = cosine_scores[0][i].item()
    print(f"  Candidate: '{fact_checked_claims[i]}' | Cosine Score: {score:.4f}")

###############################################################################
# 6) NLI Module: Evaluate Each Candidate Using roberta-large-mnli
###############################################################################
nli_tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")
nli_model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli")
nli_labels = ["contradiction", "neutral", "entailment"]

print("\nNLI evaluation for each candidate:")
for i in indices:
    premise = fact_checked_claims[i]
    hypothesis = input_claim
    # Tokenize the premise-hypothesis pair.
    nli_inputs = nli_tokenizer.encode_plus(premise, hypothesis, return_tensors="pt", truncation=True)
    nli_outputs = nli_model(**nli_inputs)
    logits = nli_outputs.logits
    probs = torch.softmax(logits, dim=1)[0]
    probs_dict = {nli_labels[j]: probs[j].item() for j in range(3)}

    print(f"\nCandidate Premise: '{premise}'")
    print(f"Hypothesis: '{hypothesis}'")
    print("NLI probabilities:", probs_dict)

    if probs[2] > probs[0] and probs[2] > probs[1]:
        print("Conclusion: This candidate supports the input claim.")
    else:
        print("Conclusion: This candidate does NOT support the input claim.")


README.md:   0%|          | 0.00/11.9k [00:00<?, ?B/s]

fever.py:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

The repository for fever contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/fever.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/311431 [00:00<?, ? examples/s]

Generating labelled_dev split:   0%|          | 0/37566 [00:00<?, ? examples/s]

...

Generating paper_dev split:   0%|          | 0/18999 [00:00<?, ? examples/s]

Generating paper_test split:   0%|          | 0/18567 [00:00<?, ? examples/s]

Fact-checked claims from FEVER:
  1. Ryan Seacrest is a person.
  2. Selena recorded music.
  3. Cyndi Lauper won the Best New Artist award at the 27th Grammy Awards in 1985.
  4. Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.
  5. Chris Hemsworth appeared in A Perfect Getaway.
  6. The Boston Celtics play their home games at TD Garden.
  7. There is a movie called The Hunger Games.
  8. History of art includes architecture, dance, sculpture, music, painting, poetry literature, theatre, narrative, film, photography and graphic arts.
  9. Roman Atwood is a content creator.
  10. Robert J. O'Neill was born April 10, 1976.

Combined embedding shape for database: (10, 1152)

Top candidates based on PCA-reduced embeddings:
  Candidate: 'The Boston Celtics play their home games at TD Garden.' | Cosine Score: 0.6836
  Candidate: 'There is a movie called The Hunger Games.' | Cosine Score: 0.5209
  Candidate: 'Chris Hemsworth appeared in A Perfect Getaway.' | Cosine Score: 0.22

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



NLI evaluation for each candidate:

Candidate Premise: 'The Boston Celtics play their home games at TD Garden.'
Hypothesis: 'The Earth is round.'
NLI probabilities: {'contradiction': 0.5456392765045166, 'neutral': 0.20503847301006317, 'entailment': 0.2493222951889038}
Conclusion: This candidate does NOT support the input claim.

Candidate Premise: 'There is a movie called The Hunger Games.'
Hypothesis: 'The Earth is round.'
NLI probabilities: {'contradiction': 0.7627103924751282, 'neutral': 0.1660517454147339, 'entailment': 0.07123780995607376}
Conclusion: This candidate does NOT support the input claim.

Candidate Premise: 'Chris Hemsworth appeared in A Perfect Getaway.'
Hypothesis: 'The Earth is round.'
NLI probabilities: {'contradiction': 0.6105901598930359, 'neutral': 0.2444082349538803, 'entailment': 0.14500166475772858}
Conclusion: This candidate does NOT support the input claim.

Candidate Premise: 'Robert J. O'Neill was born April 10, 1976.'
Hypothesis: 'The Earth is round.'
N

In [11]:
# Convert the dataset to a Pandas DataFrame for visualization
df = fever_dataset.to_pandas()

# Display the first few rows
display(df.head())

,id,label,claim,evidence_annotation_id,evidence_id,evidence_wiki_url,evidence_sentence_id
0,75397,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,92206,104971,Nikolaj_Coster-Waldau,7
1,75397,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,92206,104971,Fox_Broadcasting_Company,-1
2,150448,SUPPORTS,Roman Atwood is a content creator.,174271,187498,Roman_Atwood,1
3,150448,SUPPORTS,Roman Atwood is a content creator.,174271,187499,Roman_Atwood,3
4,214861,SUPPORTS,"History of art includes architecture, dance, s...",255136,254645,History_of_art,2


In [12]:
df.label.value_counts()

,count
label,
SUPPORTS,43
NOT ENOUGH INFO,4
REFUTES,3


#LLM

In [ ]:
import openai

# Set your OpenAI API key (you can also set this in your environment variables)
openai.api_key = "YOUR_OPENAI_API_KEY"

def fact_check_claim(claim: str) -> str:
    """
    Uses OpenAI's language model to fact-check a claim.
    The prompt instructs the model to explain its reasoning and then return a verdict.
    """
    prompt = (
        f"Please fact-check the following claim. "
        f"Provide clear supporting evidence and reasoning, "
        f"and at the end, state 'Supported' if the claim is true or 'Not Supported' if it is false.\n\n"
        f"Claim: {claim}\n\nAnswer:"
    )

    response = openai.Completion.create(
        model="text-davinci-003",  # or another appropriate model, e.g., "gpt-3.5-turbo" (using chat completions) if desired
        prompt=prompt,
        max_tokens=200,
        temperature=0,  # set to 0 for a deterministic response
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response["choices"][0]["text"].strip()

# Example claim to fact-check
claim = "The Earth is round."
result = fact_check_claim(claim)

print("Fact-checking result:")
print(result)


# Combine S-BERT and OPenAI-API

* **Specialized Database:**
You define a set of verified statements (your specialized database).

* **Retrieval:**
Using a SentenceTransformer (S‑BERT) model, we compute embeddings for the database and the input claim. Optionally, PCA reduces the embedding dimensionality. Cosine similarity is used to retrieve the top k candidates from the database.

* **Contextual Prompt:**
The retrieved evidence is concatenated into a context string. This is then inserted into a prompt that instructs the OpenAI API to fact-check the input claim using that evidence.

* **OpenAI API Fact-Checking:**
The API (here using "text-davinci-003") generates an explanation and a final verdict—either "Supported" or "Not Supported"—based on both its internal knowledge and the provided evidence.

This hybrid approach leverages both your specialized, high-quality database and the broad capabilities of an LLM from OpenAI. The benefit is improved reliability in scenarios where your database might not have complete coverage while still grounding the fact-checking process in verified information.

Computing cosine similarity lets you automatically retrieve the most relevant evidence from your database. Instead of including the entire database in the prompt—which could be overwhelming or contain irrelevant information—you use cosine similarity to rank and select only the best-matching verified statements. This focused context is then provided to the OpenAI API, which can use it to deliver a more accurate, grounded fact-checking response. Essentially, the retrieval step filters out noise, ensuring that only the most pertinent evidence influences the final verdict.

In [ ]:
import openai
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.decomposition import PCA

# Set your OpenAI API key.
openai.api_key = "YOUR_OPENAI_API_KEY"

###############################################################################
# 1) Define a Specialized Database of Verified Statements
###############################################################################
# Example specialized database (this could be a custom dataset or extracted from FEVER)
fact_checked_claims = [
    "The Earth is round.",
    "Vaccines do not cause autism.",
    "Climate change is primarily human-caused.",
    "Water boils at 100°C at sea level.",
    "The human heart has four chambers."
]

###############################################################################
# 2) Retrieve Candidate Evidence Using a SentenceTransformer Model
###############################################################################
# Load a SentenceTransformer model (S‑BERT)
sbert_model = SentenceTransformer('all-mpnet-base-v2')

# Compute embeddings for the fact-checked claims.
fact_embeddings = sbert_model.encode(fact_checked_claims, convert_to_tensor=True)

# (Optional) Apply PCA to reduce dimensionality.
# For demonstration, if you have few samples PCA might not be necessary.
n_samples, total_dim = fact_embeddings.shape
pca_dim = min(5, n_samples)  # with few samples, you can set this to n_samples or lower
pca = PCA(n_components=pca_dim)
fact_embeddings_np = fact_embeddings.cpu().numpy()
pca.fit(fact_embeddings_np)
fact_embeddings_pca = torch.tensor(pca.transform(fact_embeddings_np))

# Process the input claim.
input_claim = "The Earth is flat."
input_embedding = sbert_model.encode(input_claim, convert_to_tensor=True)
input_embedding_np = input_embedding.cpu().numpy().reshape(1, -1)
input_embedding_pca = torch.tensor(pca.transform(input_embedding_np))

# Compute cosine similarities in the PCA space.
cosine_scores = util.cos_sim(input_embedding_pca, fact_embeddings_pca)
top_k = 3  # retrieve the top 3 candidates
top_results = torch.topk(cosine_scores, k=top_k)
indices = top_results.indices[0].tolist()

# Build a context string from the retrieved candidates.
retrieved_context = "\n".join([f"- {fact_checked_claims[i]}" for i in indices])
print("Retrieved evidence from specialized database:")
print(retrieved_context)

###############################################################################
# 3) Combine Retrieved Evidence with OpenAI API for Final Fact-Checking
###############################################################################
# Construct a prompt that includes the retrieved evidence and instructs the LLM to fact-check the input claim.
prompt = (
    "We have a database of verified statements:\n"
    f"{retrieved_context}\n\n"
    "Using the above verified facts, please fact-check the following claim. "
    "Provide a detailed explanation and then at the end state 'Supported' if the claim is true or 'Not Supported' if it is false.\n\n"
    f"Claim: {input_claim}\n\nAnswer:"
)

# Call the OpenAI API with the prompt.
response = openai.Completion.create(
    model="text-davinci-003",  # or another model such as gpt-3.5-turbo via the chat completions endpoint
    prompt=prompt,
    max_tokens=200,
    temperature=0,  # 0 for deterministic responses
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

print("\nFinal Fact-Checking Result:")
print(response["choices"][0]["text"].strip())


# Online Debates

* **Specialized Database:**
A small set of verified statements is defined (e.g., "The Earth is round."). This is our trusted information source.

* **Embedding & Retrieval:**
Two SentenceTransformer models are used to generate complementary embeddings. Their outputs are concatenated, and PCA is applied to reduce dimensionality. For each claim from the debate, cosine similarity in the PCA space retrieves the top three relevant verified statements.

* **OpenAI Fact-Checking:**
The retrieved evidence is then incorporated into a prompt that instructs the OpenAI API (using a model like "text-davinci-003") to fact-check the claim. The model is asked to provide a detailed explanation and a final verdict.

* **Simulating Online Debate:**
A list of debate messages is processed one by one. For each message, the system retrieves evidence and then calls the OpenAI API to generate a fact-checking response.

In [ ]:
import openai
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.decomposition import PCA
import time

# Set your OpenAI API key
openai.api_key = "YOUR_OPENAI_API_KEY"

###############################################################################
# 1) Define a Specialized Database of Verified Statements
###############################################################################
verified_statements = [
    "The Earth is round.",
    "Vaccines do not cause autism.",
    "Climate change is primarily human-caused.",
    "Water boils at 100°C at sea level.",
    "The human heart has four chambers."
]

###############################################################################
# 2) Load Multiple SentenceTransformer Models and Precompute Embeddings
###############################################################################
model_names = [
    "all-MiniLM-L6-v2",   # Model A (384 dimensions)
    "all-mpnet-base-v2"   # Model B (768 dimensions)
]
models = [SentenceTransformer(name) for name in model_names]

def get_combined_embeddings(texts, models):
    """
    For each text, compute embeddings from each model, then concatenate.
    Returns a NumPy array of shape (num_texts, total_dim).
    """
    all_embs = []
    for text in texts:
        emb_list = [m.encode(text) for m in models]  # e.g., (384,), (768,)
        combined_emb = np.concatenate(emb_list, axis=0)  # e.g., (1152,)
        all_embs.append(combined_emb)
    return np.stack(all_embs, axis=0)

# Precompute embeddings for the verified statements.
db_embeddings = get_combined_embeddings(verified_statements, models)  # shape: (num_statements, total_dim)

# Optionally, apply PCA to reduce dimensionality.
n_samples, total_dim = db_embeddings.shape
# With a small database, choose PCA dimensions ≤ number of samples.
pca_dim = min(5, n_samples)
pca = PCA(n_components=pca_dim)
pca.fit(db_embeddings)
db_embeddings_pca = torch.tensor(pca.transform(db_embeddings))

###############################################################################
# 3) Define a Function to Retrieve Evidence from the Verified Database
###############################################################################
def retrieve_evidence(claim):
    """
    Given a claim, compute its embedding, apply the same PCA transform,
    and retrieve the top matching verified statements.
    Returns a context string of retrieved evidence.
    """
    claim_emb = get_combined_embeddings([claim], models)  # shape: (1, total_dim)
    claim_emb_pca = torch.tensor(pca.transform(claim_emb))  # shape: (1, pca_dim)
    cosine_scores = util.cos_sim(claim_emb_pca, db_embeddings_pca)
    top_k = 3
    top_results = torch.topk(cosine_scores, k=top_k)
    indices = top_results.indices[0].tolist()
    context = "\n".join([f"- {verified_statements[i]}" for i in indices])
    return context

###############################################################################
# 4) Define a Function to Fact-Check a Claim Using the OpenAI API
###############################################################################
def openai_fact_check(claim, evidence_context):
    """
    Build a prompt that includes retrieved evidence and instructs
    the LLM to fact-check the claim. Returns the API's response.
    """
    prompt = (
        "We have a database of verified statements:\n"
        f"{evidence_context}\n\n"
        "Using the above verified facts, please fact-check the following claim. "
        "Provide a detailed explanation and then at the end state 'Supported' if the claim is true or 'Not Supported' if it is false.\n\n"
        f"Claim: {claim}\n\nAnswer:"
    )
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt,
        max_tokens=200,
        temperature=0,  # deterministic response
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response["choices"][0]["text"].strip()

###############################################################################
# 5) Simulate Online Debate: Process a List of Debate Messages
###############################################################################
# Imagine these messages are coming from an online debate.
online_debate_messages = [
    "I think the Earth is flat because I've never seen any curvature.",
    "Vaccines are dangerous and cause autism!",
    "Global warming is just a hoax invented for political reasons.",
    "Water always boils at 100°C regardless of altitude.",
    "The human heart has only three chambers."
]

print("Simulated Online Debate Fact-Checking:\n")
for message in online_debate_messages:
    print("--------------------------------------------------")
    print("Debate Message:", message)
    # Step 1: Retrieve evidence from the verified database.
    evidence = retrieve_evidence(message)
    print("Retrieved Evidence:\n", evidence)
    # Step 2: Fact-check the claim using the OpenAI API.
    result = openai_fact_check(message, evidence)
    print("OpenAI Fact-Check Result:")
    print(result)
    # Pause briefly to respect API rate limits in a simulation.
    time.sleep(1)


* Below is an example that simulates integrating our hybrid fact‐checking approach into an online debate environment hosted on Discord. In this example, a Discord bot listens for messages beginning with a command (e.g., !factcheck). When a user submits a claim via that command, the bot does the following:

* Retrieves relevant evidence from a specialized database of verified statements (using multiple SentenceTransformer models plus optional PCA).
Constructs a prompt that combines the retrieved evidence with the claim.
Calls the OpenAI API to produce a fact‐checking response.
Responds in Discord with the fact-check result.
* Make sure you have installed the required packages (e.g., via pip install discord.py sentence-transformers transformers scikit-learn openai) and have set your OpenAI API key and Discord bot token.

**How This Works**

*Specialized Database & Retrieval:*
A trusted database of verified statements is defined. Two SentenceTransformer models compute complementary embeddings for these statements. Optional PCA reduces the dimensionality to make similarity calculations more efficient.
When a new claim is submitted, its embedding is computed and compared (via cosine similarity) with the database embeddings. The top candidates are then concatenated into a context string.

*OpenAI Fact-Checking:*
The retrieved evidence is incorporated into a prompt that instructs the LLM (here, using "text-davinci-003") to fact-check the claim. The API returns a detailed explanation and a verdict.

*Discord Integration:*
A Discord bot (using discord.py's commands extension) listens for a command !factcheck <claim>. When triggered, it retrieves evidence, fact-checks the claim via OpenAI API, and responds in the channel with the final result.

In [ ]:
import discord
import openai
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.decomposition import PCA
from discord.ext import commands
import os

# Set your API keys (or use environment variables)
openai.api_key = "YOUR_OPENAI_API_KEY"  # Replace with your OpenAI API key
DISCORD_TOKEN = "YOUR_DISCORD_BOT_TOKEN"  # Replace with your Discord bot token

###############################################################################
# 1) Define a Specialized Database of Verified Statements
###############################################################################
verified_statements = [
    "The Earth is round.",
    "Vaccines do not cause autism.",
    "Climate change is primarily human-caused.",
    "Water boils at 100°C at sea level.",
    "The human heart has four chambers."
]

###############################################################################
# 2) Load Multiple SentenceTransformer Models & Precompute Embeddings
###############################################################################
model_names = [
    "all-MiniLM-L6-v2",   # Model A (384 dimensions)
    "all-mpnet-base-v2"   # Model B (768 dimensions)
]
models = [SentenceTransformer(name) for name in model_names]

def get_combined_embeddings(texts, models):
    """
    For each text in 'texts', compute embeddings from each model,
    concatenate them, and return a NumPy array of shape (num_texts, total_dim).
    """
    all_embs = []
    for text in texts:
        emb_list = [m.encode(text) for m in models]  # e.g. shapes: (384,), (768,)
        combined_emb = np.concatenate(emb_list, axis=0)  # e.g. shape: (1152,)
        all_embs.append(combined_emb)
    return np.stack(all_embs, axis=0)

# Compute embeddings for the verified statements.
db_embeddings = get_combined_embeddings(verified_statements, models)  # shape: (num_statements, total_dim)

# Optionally, apply PCA to reduce dimensionality.
n_samples, total_dim = db_embeddings.shape
# For a small database, choose PCA dimensions ≤ number of samples.
pca_dim = min(5, n_samples)
pca = PCA(n_components=pca_dim)
pca.fit(db_embeddings)
db_embeddings_pca = torch.tensor(pca.transform(db_embeddings))

###############################################################################
# 3) Define a Function to Retrieve Evidence from the Verified Database
###############################################################################
def retrieve_evidence(claim):
    """
    Compute the combined embedding for the claim, apply the same PCA transform,
    and retrieve the top matching verified statements.
    Returns a string listing the top evidence.
    """
    claim_emb = get_combined_embeddings([claim], models)  # shape: (1, total_dim)
    claim_emb_pca = torch.tensor(pca.transform(claim_emb))  # shape: (1, pca_dim)
    cosine_scores = util.cos_sim(claim_emb_pca, db_embeddings_pca)
    top_k = 3
    top_results = torch.topk(cosine_scores, k=top_k)
    indices = top_results.indices[0].tolist()
    context = "\n".join([f"- {verified_statements[i]}" for i in indices])
    return context

###############################################################################
# 4) Define a Function to Fact-Check a Claim Using the OpenAI API
###############################################################################
def openai_fact_check(claim, evidence_context):
    """
    Build a prompt that includes retrieved evidence and instructs the LLM to fact-check the claim.
    Returns the API's response text.
    """
    prompt = (
        "We have a database of verified statements:\n"
        f"{evidence_context}\n\n"
        "Using the above verified facts, please fact-check the following claim. "
        "Provide a detailed explanation and then at the end state 'Supported' if the claim is true or 'Not Supported' if it is false.\n\n"
        f"Claim: {claim}\n\nAnswer:"
    )

    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt,
        max_tokens=200,
        temperature=0,  # deterministic response
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response["choices"][0]["text"].strip()

###############################################################################
# 5) Set Up the Discord Bot to Listen for Fact-Checking Commands
###############################################################################
bot = commands.Bot(command_prefix="!")

@bot.event
async def on_ready():
    print(f"Bot {bot.user} is now running!")

@bot.command(name="factcheck")
async def factcheck(ctx, *, claim: str):
    """
    Discord command: !factcheck <claim>
    The bot retrieves evidence, calls OpenAI API for fact-checking, and replies.
    """
    await ctx.send("Processing your claim...")

    # Retrieve evidence from our verified database.
    evidence = retrieve_evidence(claim)

    # Build and send the fact-check prompt to OpenAI.
    result = openai_fact_check(claim, evidence)

    # Format the response.
    response_message = (
        f"**Claim:** {claim}\n\n"
        f"**Retrieved Evidence:**\n{evidence}\n\n"
        f"**Fact-Check Result:**\n{result}"
    )

    await ctx.send(response_message)

###############################################################################
# 6) Run the Discord Bot
###############################################################################
bot.run(DISCORD_TOKEN)
